# Project: Wrangling and Analyze Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import requests
import json
from tweepy import OAuthHandler
from timeit import default_timer as timer
import tweepy
import re
import seaborn as sns
%matplotlib inline 

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
twitter_enhanced = pd.read_csv("twitter-archive-enhanced.csv")
pd.set_option("display.max_colwidth", None)